In [48]:
from BIFParser import *
file = open('data/ASIA/asia.bif')
BIF = file.readlines()
BIF = fixWhiteSpace(BIF2)
import Factor

In [34]:
nodes = parseBIF(BIF)

In [46]:
# printNodes(nodes)
nodes[1].getDist()

{(('yes', 'no'), ('no',)): (0.01, 0.99),
 (('yes', 'no'), ('yes',)): (0.05, 0.95)}

In [59]:
tempArray = [nodes[0]]
tempArray.extend(nodes[0].getParents())
factor = Factor.Factor(nodes[0].getDist(), tempArray)

In [60]:
factor.sendBelief(nodes[0])

{('e', 'n', 'o'): 0.99, ('y', 'n', 'o'): 0.01}